In [2]:
!pip install transformers

     |████████████████████████████████| 6.3 MB 4.0 MB/s eta 0:00:01     |██████████████████████████████▉ | 6.1 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 7.6 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 190 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 757 kB 5.7 MB/s eta 0:00:01


In [8]:
# import python directory
from transformers import pipeline
from tqdm import tqdm
from glob import glob
import os



In [4]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
#https://huggingface.co/facebook/bart-large-mnli

In [5]:
input_directory, output_directory = '/workspace/dataset22/', '/workspace/dataset22/'

In [28]:
# load image text
# We use some very baseline method to get a textual representation: we just use the text of the pages that contain the image.
def load_image_text(image_id):
    ret = ''
    for txt_file in glob(input_directory +'/images/' + image_id[:3] + '/' + image_id + '/*/*/*/text.txt'):
        ret += '\n\n' + open(txt_file).read()
    return ret.strip()

def load_queries():
    file_name = input_directory + 'topics-task3.xml'
    
    if not os.path.exists(file_name):
        raise ValueError(f'Could not find the file "{file_name}". Got: {glob(input_directory + "/*")}')
    
    topics = pd.read_xml(file_name)
    
    ret = pd.DataFrame()
    # https://github.com/terrier-org/pyterrier/issues/62\n",
    ret['query'] = topics['title'].apply(lambda i: "".join([x if x.isalnum() else " " for x in i]))
    ret['qid'] = topics['number']
    return ret


def determine_stance(text, query):
    labels = ["contra "+ query, "pro "+ query, "neutral "+ query]
    result = classifier(text, labels)
    # sorts the labels starting with the most likely
    if result["labels"][0].startswith("contra"):
        return "contra", result["scores"][0]
    elif result["labels"][0].startswith("pro"):
        return "pro", result["scores"][0]
    else: 
        return "neutral", result["scores"][0]
    

def detect_stance_for_image_ids_and_return_best_x_results_for_each_side(image_ids, query, x):
    # get image ids from preranking
    # get image texts based on preranking 
    pro_imgs = {}
    contra_imgs = {}
    for image_id in image_ids:
        text = load_image_text(image_id)
        # deploy model
        stance, score = determine_stance(text, query)
        print(image_id, stance, score)
        # separate pro and con texts; sort out neutral texts
        if stance == "pro":
            pro_imgs[image_id] = score
        elif stance == "contra": 
            contra_imgs[image_id] = score
        # sort according to scores
        sorted_pro = dict(sorted(pro_imgs.items(), key=lambda x:x[1], reverse=True))
        sorted_contra = dict(sorted(contra_imgs.items(), key=lambda x:x[1], reverse=True))
        # return best x results for each side
        pro_ids = []
        contra_ids = []
        i = 0
        for key in sorted_pro:
            if i < len(sorted_pro.keys()) and i < x:
                pro_ids.append(key)
            else: 
                break
            i += 1
        i = 0
        for key in sorted_contra:
            if i < len(sorted_contra.keys()) and i < x:
                contra_ids.append(key)
            else: 
                break
            i += 1
    return pro_ids, contra_ids
# not sorted right


KeyError: 0

In [9]:
queries = load_queries()  
queries.head(2)

ValueError: Could not find the file "/workspace/dataset22/topics-task3.xml". Got: ['/workspace/dataset22/images']

In [26]:
test_ids = os.listdir(input_directory +'/images/' + "I00")[0:10] #["I00", "I01","I02", "I03", "I04", "I05", "I06"]
query = ""

In [29]:
detect_stance_for_image_ids_and_return_best_x_results_for_each_side(test_ids, query, 2)

I000330ba4ea0ad13 contra 0.4679756462574005
I000d8de9c4746ee9 pro 0.3815423846244812
I0010d5b5473065fa pro 0.6040793061256409
I001652ec040f07c4 pro 0.4360259473323822
I001a318b3176d501 pro 0.379162460565567
I001b3068979b8629 pro 0.4018160402774811
I001c2de5ef28f77f pro 0.4739639461040497
I001d567f0fb443d7 neutral 0.3531545400619507
I00246a6916fec768 pro 0.3388685882091522
I0024c0097c425855 pro 0.38304683566093445


(['I0010d5b5473065fa', 'I001c2de5ef28f77f'], ['I000330ba4ea0ad13'])

In [13]:
sequence_to_classify = "Abortion is a universal right."
candidate_labels = ['pro abortion', 'contra abortion', 'neutral abortion']
classifier(sequence_to_classify, candidate_labels)


{'sequence': 'Abortion is a universal right.',
 'labels': ['pro abortion', 'neutral abortion', 'contra abortion'],
 'scores': [0.8498501777648926, 0.1026904508471489, 0.04745934531092644]}